In [ ]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

In [ ]:
TRN_cat = pd.HDFStore('/home/sirbastiano/Documenti/Python Projects/TRN/TRN/DATA/TRN_navi_catalog.h5')
TRN_cat_df = TRN_cat['df']
TRN_cat_df.head()

In [ ]:
# from numba import njit

# @njit
# def findAngles(a, b, c):
#         # applied cosine rule
#         A = np.arccos((b * b + c * c - a * a) / (2 * b * c))
#         B = np.arccos((a * a + c * c - b * b) / (2 * a * c))
#         C = np.arccos((b * b + a * a - c * c) / (2 * b * a))
#         # convert into degrees
#         A, B, C = np.rad2deg(A), np.rad2deg(B), np.rad2deg(C)
#         return A, B, C


# @njit
# def compute_K_vet(triplet):
#         a, b, c = compute_sides(triplet)
#         A, B, C = findAngles(a, b, c)
#         K_vet = np.array([A, B, C])
#         if K_vet is not None:
#             return K_vet

# @njit
# def compute_sides(triplet):
#         a = np.linalg.norm(triplet[0][0:2] - triplet[1][0:2])
#         b = np.linalg.norm(triplet[1][0:2] - triplet[2][0:2])
#         c = np.linalg.norm(triplet[2][0:2] - triplet[0][0:2])
#         return a, b, c

# def find_all_triplets(craters):

#     def Hstack(K_v, i,j,k,x1,y1,r1,x2,y2,r2,x3,y3,r3):
#         A = np.zeros(15)
#         A[0],A[1],A[2] = K_v[0],K_v[1],K_v[2]
#         A[3],A[4],A[5] = i,j,k
#         A[6],A[7],A[8] = x1,y1,r1
#         A[9],A[10],A[11] = x2,y2,r2
#         A[12],A[13],A[14] = x3,y3,r3
#         return A
    
#     def eu_dist(x, y):
#         x1, y1 = x[0], x[1]
#         x2, y2 = y[0], y[1]
#         result = ((((x2 - x1)**2) + ((y2-y1)**2))**0.5)
#         return result

#     def concat(a,b,c):
#         A = np.zeros((3,3))
#         A[0] = a
#         A[1] = b
#         A[2] = c
#         return A

#     # Input: np.array craters
#     # Output: all triplets
#     N = craters.shape[0]
#     ender = N*N*N
#     K = np.zeros((ender,15))
#     lister = 0
#     for i in range(N):
#         for j in range(N):
#             for k in range(N):
#                 if (i!=j) & (j!=k):
#                     a =craters[i]
#                     b =craters[j]
#                     c =craters[k]
#                     triplet = concat(a,b,c)
#                     x1,y1,r1 = a[0],a[1],a[2]
#                     x2,y2,r2 = b[0],b[1],b[2]
#                     x3,y3,r3 = c[0],c[1],c[2]
                    
#                     C = np.zeros(2) # centroid
#                     C[0] = (x1+x2+x3)/3
#                     C[1] = (y1+y2+y3)/3

#                     P1,P2,P3 = np.zeros(2), np.zeros(2), np.zeros(2)
#                     P1[0]=x1
#                     P1[1]=y1
#                     P2[0]=x2
#                     P2[1]=y2
#                     P3[0]=x3
#                     P3[1]=y3

#                     d1,d2,d3 = eu_dist(P1,C), eu_dist(P2,C), eu_dist(P3,C)
#                     d_i,d_j,d_k = d1/r1, d2/r2, d3/r3


#                     try:
#                         K_v = compute_K_vet(triplet)
#                         K[lister] = Hstack(K_v, d_i,d_j,d_k,x1,y1,r1,x2,y2,r2,x3,y3,r3)
#                     except ZeroDivisionError: pass

#                 lister+=1
#     return K[ np.all(K !=0, axis=1) ]

In [ ]:
# LOAD ALL IMAGES:
dt = 10
dict = {}
for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
    txt = img             # stringa
    t = txt.split('_')[1] # numero
    dict[t] = txt

In [ ]:
def verify(save, tol = 3):
    c1_t1 = [save[0].x1, save[0].y1, save[0].r1]
    c2_t1 = [save[0].x2, save[0].y2, save[0].r3]
    c3_t1 = [save[0].x3, save[0].y2, save[0].r3]

    c1_t2 = [save[1].x1, save[1].y1, save[1].r1]
    c2_t2 = [save[1].x2, save[1].y2, save[1].r3]
    c3_t2 = [save[1].x3, save[1].y2, save[1].r3]

    centroid_1 = [(c1_t1[0]+c2_t1[0]+c3_t1[0])/3, (c1_t1[1]+c2_t1[1]+c3_t1[1])/3]
    centroid_2 = [(c1_t2[0]+c2_t2[0]+c3_t2[0])/3, (c1_t2[1]+c2_t2[1]+c3_t2[1])/3]

    delta_x = abs(centroid_1[0]-centroid_2[0])
    delta_y = abs(centroid_1[1]-centroid_2[1])

    if c1_t1[2]-c1_t2[2] < tol: # Se il primo cratere è il primo cratere bis
        if (abs(abs(c1_t1[0]-c1_t2[0])-delta_x) < tol) & (abs(abs(c1_t1[1]-c1_t2[1])-delta_y) < tol):
            return True
    elif c1_t1[2]-c2_t2[2] < tol: # primo - secondo
        if (abs(abs(c1_t1[0]-c2_t2[0])-delta_x) < tol) & (abs(abs(c1_t1[1]-c2_t2[1])-delta_y) < tol):
            return True
    elif c1_t1[2]-c2_t2[2] < tol: # primo - terzo
        if (abs(abs(c1_t1[0]-c3_t2[0])-delta_x) < tol) & (abs(abs(c1_t1[1]-c3_t2[1])-delta_y) < tol):
            return True
    else: return False

# ODOMETRY

In [ ]:
idx = 55
tol = 5
# IMG1:
img1=cv2.imread(dict[str(idx+1)])
craters_det1 = detect(img1)
# IMG2:
img2=cv2.imread(dict[str(idx+2)])
craters_det2 = detect(img2)

# Pandas DataFrame:
df_craters_det1 = sort_mat(craters_det1)
df_craters_det2 = sort_mat(craters_det2)
# Find all triplets:
H1 = find_all_triplets(craters_det1)
H2 = find_all_triplets(craters_det2)
QUERY1= pd.DataFrame(H1, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
QUERY2= pd.DataFrame(H2, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
# remove wrong triplet:
QUERY1 =QUERY1[ (QUERY1.Angle1 > 10) & (QUERY1.Angle2 > 10) & (QUERY1.Angle3 > 10) ].reset_index(drop=True)
QUERY2 =QUERY2[ (QUERY2.Angle1 > 10) & (QUERY2.Angle2 > 10) & (QUERY2.Angle3 > 10) ].reset_index(drop=True)
##########################################
# SEARCH
##########################################
found = False
for idx in range(QUERY1.shape[0]):
    Angle1_Q1 = QUERY1.Angle1[idx]
    Angle2_Q1 = QUERY1.Angle2[idx]
    Angle3_Q1 = QUERY1.Angle3[idx]
    d1_Q1 = QUERY1.des1[idx]
    d2_Q1 = QUERY1.des2[idx]
    d3_Q1 = QUERY1.des3[idx]

    SEARCH = QUERY2[ ((abs(QUERY2.Angle1 - Angle1_Q1) < tol) & (abs(QUERY2.Angle2 - Angle2_Q1) < tol) & (abs(QUERY2.Angle3 - Angle3_Q1) < tol))   \
                      |  ((abs(QUERY2.Angle1 - Angle2_Q1) < tol) & (abs(QUERY2.Angle2 - Angle3_Q1) < tol) & (abs(QUERY2.Angle3 - Angle1_Q1) < tol)) \
                      |  ((abs(QUERY2.Angle1 - Angle3_Q1) < tol) & (abs(QUERY2.Angle2 - Angle1_Q1) < tol) & (abs(QUERY2.Angle3 - Angle2_Q1) < tol))]

    SEARCH = SEARCH  [ ((abs(QUERY2.des1- d1_Q1) < tol) & (abs(QUERY2.des2- d2_Q1) < tol) & (abs(QUERY2.des3- d3_Q1) < tol)) \
                     | ((abs(QUERY2.des1- d2_Q1) < tol) & (abs(QUERY2.des2- d3_Q1) < tol) & (abs(QUERY2.des3- d1_Q1) < tol)) \
                     | ((abs(QUERY2.des1- d3_Q1) < tol) & (abs(QUERY2.des2- d1_Q1) < tol) & (abs(QUERY2.des3- d2_Q1) < tol)) ]
    if SEARCH.shape[0] != 0:
        print('Maybe I found!')
        save = [QUERY1.iloc[idx], SEARCH.iloc[0]]
        switch = verify(save, tol = 10)
        if switch:
            print('FOUND!')
            found = True
            break
        else:
            continue
if found:
    a = save[0]
    b = save[1]
    crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
    crts2 = np.vstack([[b.x1,b.y1,b.r1],[b.x2,b.y2,b.r2],[b.x3,b.y3,b.r3]])

    plt.figure(dpi=200)
    plt.subplot(121)
    cp1 = img1.copy()
    IMG1 =  img_plus_crts(cp1, crts1, color="red")
    plt.imshow(IMG1)

    plt.subplot(122)
    cp2 = img2.copy()
    IMG2 =  img_plus_crts(cp2, crts2, color="red")
    plt.imshow(IMG2)
    plt.show()
else:
    print("Match can't find any triplets.")

# CATALOG NAVI

In [ ]:
def verifica(SEARCH, filename):
    t = filename.split('_')[-1].split('.')
    longy = t[0]+'.'+t[1]
    longy = float(longy)    
    for triplet in SEARCH.iloc:
            if (triplet.lat1 < 5) & (triplet.lat1 > -5) & (triplet.lon1 < longy+5) & (triplet.lon1 > longy-5):
                return True
            else: return False


def print_res(SEARCH, filename):
    t = filename.split('_')[-1].split('.')
    longy = t[0]+'.'+t[1]
    longy = float(longy)    
    for triplet in SEARCH.iloc:
            if (triplet.lat1 < 2) & (triplet.lat1 > -2) & (triplet.lon1 < longy+3) & (triplet.lon1 > longy-3):
                print(triplet)

In [ ]:
for idx in range(5):
    # IMG1:
    filename = dict[str(idx+1)]
    img1=cv2.imread(filename)
    craters_det1 = detect(img1)

    # Pandas DataFrame:
    df_craters_det1 = sort_mat(craters_det1)
    # Find all triplets:
    H1 = find_all_triplets(craters_det1)
    QUERY1= pd.DataFrame(H1, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
    # remove wrong triplet:
    QUERY1 =QUERY1[ (QUERY1.Angle1 > 10) & (QUERY1.Angle2 > 10) & (QUERY1.Angle3 > 10) ].reset_index(drop=True)
    QUERY2 = TRN_cat_df

    q1 = QUERY1
    q2 = QUERY2

    q1=q1.astype({'Angle1':'int32','Angle2':'int32','Angle3':'int32','des1':'int32','des2':'int32','des3':'int32'    })
    q2=q2.astype({'Angle1':'int32','Angle2':'int32','Angle3':'int32','des1':'int32','des2':'int32','des3':'int32'    })
    S = pd.merge(q1,q2, on=['Angle1','Angle2','Angle3'],   how='inner', indicator=False)
    if verifica(S, filename):
         print('\r')
         print('FOUND')
         break

    q1=swap_df_columns('Angle1','Angle2',q1)
    S = pd.merge(q1,q2, on=['Angle1','Angle2','Angle3'],   how='inner', indicator=False)
    if verifica(S, filename):
         print('FOUND')
         break

    q1=swap_df_columns('Angle2','Angle1',q1) #reverse
    q1=swap_df_columns('Angle1','Angle3',q1)
    S = pd.merge(q1,q2, on=['Angle1','Angle2','Angle3'],   how='inner', indicator=False)
    if verifica(S, filename):
         print('FOUND')
         break

    q1=swap_df_columns('Angle3','Angle1',q1) #reverse
    q1=swap_df_columns('Angle2','Angle3',q1)
    S = pd.merge(q1,q2, on=['Angle1','Angle2','Angle3'],   how='inner', indicator=False)
    if verifica(S, filename):
         print('FOUND')
         break

    q1=swap_df_columns('Angle3','Angle2',q1) #reverse
    q1=swap_df_columns('Angle1','Angle3',q1)
    q1=swap_df_columns('Angle2','Angle3',q1)
    S = pd.merge(q1,q2, on=['Angle1','Angle2','Angle3'],   how='inner', indicator=False)
    if verifica(S, filename):
         print('FOUND')
         break
    
     
    

In [65]:
idx = 0

filename = dict[str(idx+1)]
img1=cv2.imread(filename)
craters_det1 = detect(img1)
# Pandas DataFrame:
df_craters_det1 = sort_mat(craters_det1)
# Find all triplets:
H1 = find_all_triplets(craters_det1)
QUERY1= pd.DataFrame(H1, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
# remove wrong triplet:
QUERY1 =QUERY1[ (QUERY1.Angle1 > 10) & (QUERY1.Angle2 > 10) & (QUERY1.Angle3 > 10) ].reset_index(drop=True)
QUERY2 = TRN_cat_df
q1 = QUERY1.sort_values(by='Angle1')
q2 = QUERY2.sort_values(by='Angle1')

S = pd.merge_asof(q1, q2, on='Angle1', direction='nearest', tolerance = 100)


 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


In [66]:
S

,Angle1,Angle2_x,Angle3_x,des1_x,des2_x,des3_x,x1,y1,r1_x,x2,...,des3_y,lon1,lat1,r1_y,lon2,lat2,r2_y,lon3,lat3,r3_y
0,10.003081,51.862331,118.134588,19.356883,11.605013,26.253454,573.5,606.5,11.00,465.0,...,7.441244,-121.083854,9.258146,3.201548,-120.882655,8.228711,3.360335,-120.309874,7.294614,4.340339
1,10.003081,118.134588,51.862331,11.605013,19.356883,26.253454,465.0,630.5,13.25,573.5,...,7.441244,-121.083854,9.258146,3.201548,-120.882655,8.228711,3.360335,-120.309874,7.294614,4.340339
2,10.011417,111.788222,58.200361,11.467438,17.246586,22.768650,547.5,549.5,15.50,465.0,...,8.994742,-50.935612,-40.302602,4.785999,-51.065669,-38.989722,3.676004,-50.363192,-37.063022,5.979731
3,10.011417,58.200361,111.788222,17.246586,11.467438,22.768650,465.0,630.5,13.25,547.5,...,8.994742,-50.935612,-40.302602,4.785999,-51.065669,-38.989722,3.676004,-50.363192,-37.063022,5.979731
4,10.055176,128.761221,41.183603,17.684598,17.223419,22.813803,137.5,248.5,13.50,134.5,...,16.608201,-163.242804,1.443132,3.298767,-161.327637,2.566386,7.292790,-160.380439,3.570815,3.001518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5893,158.764814,10.081921,11.153265,10.467699,7.830888,0.897979,455.0,778.0,12.00,547.5,...,2.435222,-58.265311,-34.729272,2.561515,-55.079006,-31.062564,2.696543,-56.581718,-33.470079,4.825064
5894,158.994916,10.303064,10.702020,3.169600,11.294822,1.203935,49.0,652.0,74.00,506.0,...,1.063629,73.867240,-31.313830,7.106199,76.565122,-34.582946,17.953491,76.086538,-33.517752,19.767361
5895,158.994916,10.702020,10.303064,11.294822,3.169600,1.203935,506.0,739.0,20.50,49.0,...,1.063629,73.867240,-31.313830,7.106199,76.565122,-34.582946,17.953491,76.086538,-33.517752,19.767361
5896,159.149754,10.540136,10.310110,11.887076,15.537033,3.192766,290.0,654.0,24.00,827.0,...,6.885825,160.623778,37.815191,18.921869,163.271381,36.003729,3.324592,161.060107,37.686589,3.463196


In [56]:
S = S[['Angle1','Angle2_y','Angle3_y']]
S.rename(columns={'Angle2_y':'Angle2','Angle3_y':'Angle3'}, inplace=True)

TypeError: rename() got an unexpected keyword argument 'tolerance'

In [49]:
df = S
S = pd.merge_asof(q1.sort_values(by='Angle2'), df.sort_values(by='Angle2'), on='Angle2', direction='nearest')

In [53]:
# S = S[['Angle1_y','Angle2','Angle3_y']]
# S.rename(columns={'Angle1_y':'Angle1','Angle3_y':'Angle3'}, inplace=True)
df = S
S = pd.merge_asof(q1.sort_values(by='Angle3'), df.sort_values(by='Angle3'), on='Angle3', direction='nearest')

In [54]:
S

,Angle1_x,Angle2_x,Angle3,des1,des2,des3,x1,y1,r1,x2,y2,r2,x3,y3,r3,Angle1_y,Angle2_y
0,51.862331,118.134588,10.003081,11.605013,26.253454,19.356883,465.0,630.5,13.25,137.5,248.5,13.50,573.5,606.5,11.00,34.058692,135.994778
1,118.134588,51.862331,10.003081,19.356883,26.253454,11.605013,573.5,606.5,11.00,137.5,248.5,13.50,465.0,630.5,13.25,34.058692,135.994778
2,111.788222,58.200361,10.011417,17.246586,22.768650,11.467438,465.0,630.5,13.25,329.5,28.0,17.25,547.5,549.5,15.50,34.058692,135.994778
3,58.200361,111.788222,10.011417,11.467438,22.768650,17.246586,547.5,549.5,15.50,329.5,28.0,17.25,465.0,630.5,13.25,34.058692,135.994778
4,41.183603,128.761221,10.055176,17.684598,22.813803,17.223419,137.5,248.5,13.50,794.5,761.5,26.50,134.5,27.5,22.50,80.758483,89.158677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5893,11.153265,10.081921,158.764814,10.467699,0.897979,7.830888,455.0,778.0,12.00,525.0,666.5,17.75,547.5,549.5,15.50,11.076218,10.240644
5894,10.303064,10.702020,158.994916,11.294822,1.203935,3.169600,506.0,739.0,20.50,290.0,654.0,24.00,49.0,652.0,74.00,11.076218,10.240644
5895,10.702020,10.303064,158.994916,3.169600,1.203935,11.294822,49.0,652.0,74.00,290.0,654.0,24.00,506.0,739.0,20.50,11.076218,10.240644
5896,10.540136,10.310110,159.149754,15.537033,3.192766,11.887076,827.0,458.0,18.50,573.5,606.5,11.00,290.0,654.0,24.00,10.003081,10.469516


In [ ]:
# #SEARCH
# tol = 5

# QUERY1_arr = np.array(QUERY1)
# for i in range(QUERY1_arr.shape[0]):
#     row = QUERY1_arr[i]
#     printProgressBar(i+1,QUERY1.shape[0], printEnd='')

#     Angle1_Q1, Angle2_Q1, Angle3_Q1 = row[0],row[1],row[2] 
#     d1_Q1, d2_Q1, d3_Q1 = row[3], row[4], row[5] 
#     SEARCH = QUERY2[ ((abs(QUERY2.Angle1 - Angle1_Q1) < tol) & (abs(QUERY2.Angle2 - Angle2_Q1) < tol) & (abs(QUERY2.Angle3 - Angle3_Q1) < tol))   \
#                           |  ((abs(QUERY2.Angle1 - Angle2_Q1) < tol) & (abs(QUERY2.Angle2 - Angle3_Q1) < tol) & (abs(QUERY2.Angle3 - Angle1_Q1) < tol)) \
#                           |  ((abs(QUERY2.Angle1 - Angle3_Q1) < tol) & (abs(QUERY2.Angle2 - Angle1_Q1) < tol) & (abs(QUERY2.Angle3 - Angle2_Q1) < tol))]
#     SEARCH = SEARCH  [     ((abs(QUERY2.des1- d1_Q1) < tol) & (abs(QUERY2.des2- d2_Q1) < tol) & (abs(QUERY2.des3- d3_Q1) < tol)) \
#                          | ((abs(QUERY2.des1- d2_Q1) < tol) & (abs(QUERY2.des2- d3_Q1) < tol) & (abs(QUERY2.des3- d1_Q1) < tol)) \
#                          | ((abs(QUERY2.des1- d3_Q1) < tol) & (abs(QUERY2.des2- d1_Q1) < tol) & (abs(QUERY2.des3- d2_Q1) < tol)) ]


#     if SEARCH.shape[0] != 0:
#         saved_idx = idx
#         if verifica(SEARCH, filename):
#             SEARCH
#             break

In [ ]:
# # Searching:
# count,d, found = 0, 10, False
# # for i in QUERY1.iloc:
# for idx in range(QUERY1.shape[0]):
#     if found:
#         break

#     i = QUERY1.iloc[idx]
#     count+=1
#     printProgressBar(count, QUERY1.shape[0], printEnd='')
    
#     for idy in range(QUERY2.shape[0]):
#         j = QUERY2.iloc[idy]
#         a1,a2,a3 = i.Angle1,i.Angle2,i.Angle3
#         b1,b2,b3 = j.Angle1,j.Angle2,j.Angle3
#         x1,x2,x3 = i.r1,i.r2,i.r3
#         y1,y2,y3 = j.r1,j.r2,j.r3
# #       switch
#         d1 = abs(a1-b1)+abs(a2-b2)+abs(a3-b3)
#         d2 = abs(a1-b2)+abs(a2-b3)+abs(a3-b1)
#         d3 = abs(a1-b3)+abs(a2-b1)+abs(a3-b2)
# #       switch
#         r1 = abs(x1-y1)+abs(x2-y2)+abs(x3-y3)
#         r2 = abs(x1-y2)+abs(x2-y3)+abs(x3-y1)
#         r3 = abs(x1-y3)+abs(x2-y1)+abs(x3-y2)

#         d = min(d1,d2,d3)
#         r = min(r1,r2,r3)
#         if (d < 2) & (r < 5):
#             print('\nFOUNDED')
#             save = [idx, idy]
#             print(save)
#             print(d,r)
#             found = True
#             break
# print('\n')
# if found:
#     a = QUERY1.iloc[save[0]]
#     b = QUERY2.iloc[save[1]]
#     crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
#     crts2 = np.vstack([[b.x1,b.y1,b.r1],[b.x2,b.y2,b.r2],[b.x3,b.y3,b.r3]])

#     plt.figure(dpi=200)
#     plt.subplot(121)
#     cp1 = img1.copy()
#     IMG1 =  img_plus_crts(cp1, crts1, color="red")
#     plt.imshow(IMG1)

#     plt.subplot(122)
#     cp2 = img2.copy()
#     IMG2 =  img_plus_crts(cp2, crts2, color="red")
#     plt.imshow(IMG2)
#     plt.show()
# else:
#     print("Match can't find any triplets.")

In [ ]:
# def find_query(df_craters):
#     STORED = []
#     c=0
#     M=df_craters.shape[0]
#     QUERY = np.zeros((M**2,15))
#     for i in range(M):
#         for j in range(M):
#             if i != j:
#                 triplet, HP, PICKS = find_triplet(df_craters, idx_a = i, idx_b=j)
#                 Names = [triplet[0].name,triplet[1].name,triplet[2].name]
#                 names = set(Names)
#                 if names not in STORED:
#                     STORED.append(names)
#                     K_v = compute_K_vet(triplet)
#                     c1 = [triplet[0].x,triplet[0].y,triplet[0].r]
#                     c2 = [triplet[1].x,triplet[1].y,triplet[1].r]
#                     c3 = [triplet[2].x,triplet[2].y,triplet[2].r]
#                     QUERY[c] = np.hstack([K_v, Names,c1,c2,c3])
#                     c+=1
#                 else: continue

#         printProgressBar(i+1, df_craters.shape[0], printEnd='')
#     QUERY = pd.DataFrame(QUERY, columns=['Angle1','Angle2','Angle3','ctr1','ctr2','ctr3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
#     return QUERY

In [ ]:
# # Searching:
# count,d, found = 0, 10, False
# # for i in QUERY1.iloc:
# for idx in range(QUERY1.shape[0]):
#     if found:
#         break

#     i = QUERY1.iloc[idx]
#     count+=1
#     printProgressBar(count, QUERY1.shape[0], printEnd='')
    
#     for idy in range(QUERY2.shape[0]):
#         j = QUERY2.iloc[idy]
#         a1,a2,a3 = i.Angle1,i.Angle2,i.Angle3
#         b1,b2,b3 = j.Angle1,j.Angle2,j.Angle3
#         x1,x2,x3 = i.r1,i.r2,i.r3
#         y1,y2,y3 = j.r1,j.r2,j.r3
# #       switch
#         d1 = abs(a1-b1)+abs(a2-b2)+abs(a3-b3)
#         d2 = abs(a1-b2)+abs(a2-b3)+abs(a3-b1)
#         d3 = abs(a1-b3)+abs(a2-b1)+abs(a3-b2)
# #       switch
#         r1 = abs(x1-y1)+abs(x2-y2)+abs(x3-y3)
#         r2 = abs(x1-y2)+abs(x2-y3)+abs(x3-y1)
#         r3 = abs(x1-y3)+abs(x2-y1)+abs(x3-y2)

#         d = min(d1,d2,d3)
#         r = min(r1,r2,r3)
#         if (d < 2) & (r < 5):
#             print('\nFOUNDED')
#             save = [idx, idy]
#             print(save)
#             print(d,r)
#             found = True
#             break
# print('\n')
# if found:
#     a = QUERY1.iloc[save[0]]
#     b = QUERY2.iloc[save[1]]
#     crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
#     crts2 = np.vstack([[b.x1,b.y1,b.r1],[b.x2,b.y2,b.r2],[b.x3,b.y3,b.r3]])

#     plt.figure(dpi=200)
#     plt.subplot(121)
#     cp1 = img1.copy()
#     IMG1 =  img_plus_crts(cp1, crts1, color="red")
#     plt.imshow(IMG1)

#     plt.subplot(122)
#     cp2 = img2.copy()
#     IMG2 =  img_plus_crts(cp2, crts2, color="red")
#     plt.imshow(IMG2)
#     plt.show()
# else:
#     print("Match can't find any triplets.")

In [ ]:
# def matching(crat_det, crat_cat):
    
#     mat1 = crat_det
#     mat2 = crat_cat

#     MATCHER = []
#     for i in range(mat1.shape[0]):
#         for j in range(mat2.shape[0]):
#             f1 = mat1[i, :]
#             f2 = mat2[j, :]
#             tmp = sq_dif(f1,f2)
#             TMP = [i, j, tmp]
#             MATCHER.append(TMP)
#     flag = np.zeros(2)
#     for elem in MATCHER:
#         if elem[2] < 100:
#             match = [elem[0],elem[1]] 
#             flag = np.vstack((flag, match))
#     flag = flag[1:,:]
#     flag = np.array(flag).astype(int)
#     return flag


In [ ]:
# flag = np.zeros(2)
# for elem in MATCHER:
#     if elem[2] < 100:
#         match = [elem[0],elem[1]] 
#         flag = np.vstack((flag, match))
# flag = flag[1:,:]
# flag = np.array(flag).astype(int)
# flag

In [ ]:
# # Stack the images
# img1 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_1_Lat_0_Long_-147.8129.jpg')
# img2 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_2_Lat_0_Long_-147.7069.jpg')
# img_matched = np.hstack((img1,img2)).astype(int)
# plt.imshow(img_matched, cmap='gray')
# plt.axis(False)
# plt.grid(False)

In [ ]:
# cv2.arrowedLine(img_matched,start,end,(255, 0, 0) , 3)

In [ ]:
# center_coordinates = start
# radius = 500
# color = (0,0,255)
# thickness = 6
# cv2.circle(img_matched, center_coordinates, radius, color, thickness)